In [1]:
# importing packages
import pandas as pd
import re

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
#Just in case if the element is yet to be displayed
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [2]:
# replace variables here.
url = "https://www.linkedin.com/jobs/search/?f_TPR=r604800&geoId=102454443&keywords=&location=Singapore&sortBy=DD"
no_of_jobs = 1000

In [3]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome("C:/Users/Johan Setiawan/Downloads/chromedriver_win32/chromedriver.exe")
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [4]:
# Change this to 5 or lesser if your internet is good
SCROLL_PAUSE_TIME = 10

# Element that is used to scroll down
sd_element = '/html/body/main/div/section/button'

# WebDriverWait
#WebDriverWait wd_wait = new WebDriverWait(driver, 10)


# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        #wd_wait.until(ExpectedCondidtions.visibilityOf(sd_element))
        driver.find_element_by_xpath(sd_element).click()
        break
    last_height = new_height

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/div/section/button"}
  (Session info: chrome=90.0.4430.93)


In [ ]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

In [ ]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
foreigner = []

# function to check if a word is inside a paragraph
def contains_word(s, w):
    return f' {w} ' in f' {s} '

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

# for loop for job description
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(5)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    # find the show more button and click
    ## !!not sure why these 2 lines don't work
    #showmore_btn = driver.find_element_by_css_selector(".show-more-less-html__button show-more-less-html__button--more")
    #showmore_btn.click()
    job_desc.append(job_descs)
    
    # The amount of words can be added, but will take longer time
    words_searched = ['singaporean', 'singapore citizen', 'PR']
    
    #To find the words_searched in the job title
    jt_foreigner = 'T'
    jt = job_titles.lower()
    
    for word in words_searched:
        jt_foreigner = contains_word(jt, word)
        if jt_foreigner == True:
            jt_foreigner = 'F'
            break
            
    
    jd_foreigner = 'T'
    jd = job_descs.lower()
    
    #To find the words_searched in the job description
    for word in words_searched:
        jd_foreigner = contains_word(jd, word)
        if jd_foreigner == True:
            jd_foreigner = 'F'
            break
            
    foreinger = 'F'
    
    if(jt_foreigner == 'F' or jd_foreigner == 'F'):
        foreigner = 'F'
    else:
        foreigner = 'T'
    
    x = x+1

In [ ]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(foreigner))

In [ ]:
# creating a dataframe
job_data = pd.DataFrame({
    'Job ID': job_id,
    'Date': post_date,
    'Company Name': company_name,
    'Post': post_title,
    'Location': job_location,
    'Description': job_desc,
    'foreigner': foreigner,
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

In [ ]:
job_data.to_csv('LinkedIn Job Data.csv', index=0)